# 🧬 User Manual: Quantum Protein Folding

Welcome to the comprehensive guide for the **Quantum Protein Folding** project.

Our implementation is **inspired by and extends** the methods described in  
[*Protein Folding Problem: A Quantum Approach*](https://arxiv.org/pdf/1908.02163), and this repository: [quantum-protein-folding-qiskit](https://github.com/qiskit-community/quantum-protein-folding)


### 🎯 Objectives
By the end of this notebook, you will understand:
1. **The Abstraction**: How we map a biological problem to a quantum-computing approach.
2. **The Hamiltonians**: How we penalize "impossible" folds (like breaking the chain) and reward "good" folds (hydrophobic contacts).
3. **The Algorithm**: How VQE (Variational Quantum Eigensolver) finds the optimal structure.

### ⚠️ Constraints & Limits
> **Chain Length Constraints:**
> *   **Minimum Length:** The simulation requires a chain length of **N ≥ 5**. This is enforced by the `MIN_CHAIN_LENGTH` constant.
> *   **Maximum Length:** While the codebase theoretically supports unlimited chain lengths - we recommend to choose chains of up to **N ≈ 9-12** since the computational resources required scale exponentially.
> *   **Supported Amino Acid Symbols:** We support the basic `20` one-letter symbols of amino acids, but this model enables you to provide your own symbols and their respective pairwise energies by modifying `mj_matrix.txt` under the `resources` directory.
>   
> *Recommendation:* For this demo, we stick to **N=7 (APRLRFY)** to ensure quick execution and to mimic the results presented in the [reference solution](https://github.com/qiskit-community/quantum-protein-folding).

For more information regarding the assumptions and constants setup please visit [official documentation](https://qfold-thesis.github.io/quantum-protein-folding/).

## ⚙️ Step 0: Environment Setup

Before continuing, please make sure that you've completed the installation steps required in the project's [`README`](https://github.com/QFold-Thesis/quantum-protein-folding?tab=readme-ov-file#-installation).

Because this notebook resides in the `docs/` directory, we must manually tell Python where to find the source code (`src/`).

In [10]:
import sys
import os
from pathlib import Path

# Locate the project root relative to this notebook
current_dir = Path(os.getcwd())
project_src = current_dir.parent / "src"

# Inject into sys.path
if str(project_src) not in sys.path:
    sys.path.append(str(project_src))
    print("✅ System Path Updated")

# Import project utilities
import logging
from logger import get_logger
from IPython.display import Image, display

logger = get_logger()
logger.disabled = True

---
# 🟢 Part 1: Automated Workflow (The "Easy" Way)

For most users, the `setup_utils` module provides a "Battery Included" experience. It handles the orchestration of classes for you.

### 🧪 1. Define the Protein Sequence

We define the protein as a string of characters. The validity of these characters depends on the **Interaction Model** you are using (defined in `constants.py`).

| Model | Description |
| :--- | :--- |
| **MJ** | Uses realistic Miyazawa-Jernigan contact energies. |
| **HP** | Simplified model. Hydrophobic beads attract each other, polar beads are neutral. |

**Pro-Tip:** If you see an `UnsupportedAminoAcidSymbolError`, check if your sequence matches your selected model's data.

In [11]:
from constants import EMPTY_SIDECHAIN_PLACEHOLDER
from utils.setup_utils import setup_folding_system

# Define your sequence here
main_chain = "APRLRFY"
side_chain = EMPTY_SIDECHAIN_PLACEHOLDER * len(main_chain)

print(f"🧬 Analyzing Sequence: {main_chain}")

# The setup function automatically:
# 1. Instantiates the Protein object
# 2. Selects the Interaction Model (MJ/HP)
# 3. Pre-calculates the Contact Map (Which beads are neighbors)
protein, interaction, contact_map, distance_map = setup_folding_system(
    main_chain=main_chain, 
    side_chain=side_chain
)

🧬 Analyzing Sequence: APRLRFY


### 🔨 2. Construct the Hamiltonian

The **Hamiltonian** ($H$) is the heart of the simulation. It represents the total energy of the system as a sum of terms:

$$ H_{total} = H_{interaction} + H_{backbone} + H_{chirality} + H_{overlap} $$

- **$H_{interaction}$**: Negative energy (good!) when correct amino acids touch.
- **$H_{backbone}$**: Large positive penalty (bad!) if the chain breaks.
- **$H_{chirality}$**: Penalty if the chain makes impossible turns.
- **$H_{overlap}$**: Penalty if two beads are in the same spot.

**Compression:** We also run a compression step to remove Pauli terms that are all-identity.

In [12]:
from utils.setup_utils import build_and_compress_hamiltonian

original_h, compressed_h = build_and_compress_hamiltonian(
    protein=protein,
    interaction=interaction,
    contact_map=contact_map,
    distance_map=distance_map,
)

print(f"📉 Optimization Space Reduced: {original_h.num_qubits} qubits -> {compressed_h.num_qubits} qubits")

📉 Optimization Space Reduced: 48 qubits -> 9 qubits


### 🚀 3. Solve with VQE

We use the **Variational Quantum Eigensolver (VQE)** to find the configuration that minimizes the Hamiltonian.

**How it works:**
1. The Quantum Computer (Ansatz) prepares a guess state.
2. We measure the energy of that guess.
3. A Classical Optimizer (COBYLA) tweaks the parameters to lower the energy.
4. Repeat until convergence.

In [13]:
from utils.setup_utils import setup_vqe_optimization, run_vqe_optimization, setup_result_analysis

# 1. Setup VQE (Ansatz + Optimizer)
vqe, counts, values = setup_vqe_optimization(num_qubits=compressed_h.num_qubits)

# 2. Run Optimization
print("⏳ VQE Running... (This may take a moment)")
raw_results = run_vqe_optimization(vqe=vqe, hamiltonian=compressed_h)

print(f"✅ Optimization Complete!")
print(f"   Minimum Energy Found: {raw_results.eigenvalue.real:.4f}")

⏳ VQE Running... (This may take a moment)
✅ Optimization Complete!
   Minimum Energy Found: -1.3971


### 📊 4. Analyze & Visualize

Finally, we decode the quantum result. The VQE returns a bitstring that represents a sequence of **turns** in the lattice (e.g., "Right, Up, Left"). We convert this turn sequence into Cartesian coordinates (X, Y, Z) and plot it.

In [14]:
print("Interpreting and visualizing results... (this might take a while)")

# Interpret results
result_interpreter, result_visualizer = setup_result_analysis(
    raw_results=raw_results,
    protein=protein,
    vqe_iterations=counts,
    vqe_energies=values,
)

# Save result interpretation to files
result_interpreter.dump_results_to_files()

# Generate 3D visualizations
result_visualizer.visualize_3d()
result_visualizer.visualize_2d()
result_visualizer.generate_3d_gif()

print(f"If you wish to see the files and visualizations for this exact run - please check: {result_visualizer._dirpath}")


Interpreting and visualizing results... (this might take a while)
If you wish to see the files and visualizations for this exact run - please check: ...\quantum-protein-folding\output\results\2025_12_07-01_02_39-APRLRFY-_______


If you wish to see how we handled 3D data visualization, you can check out:
- [3D Interactive Plot](https://qfold-thesis.github.io/quantum-protein-folding/assets/interactive_3d_visualization.html)
- [3D Rotating Conformation GIF](https://raw.githubusercontent.com/QFold-Thesis/quantum-protein-folding/gh-pages/assets/rotating_3d_visualization.gif)


---
# 🔴 Part 2: Manual Workflow (The "Deep Dive")

If you want to modify specific parameters (like changing the **Interaction Model** from MJ to HP, or tweaking the **Penalty Values**), you should use the manual workflow.

### Step 1: Select Interaction Model

Uncomment the model you wish to use below. Note that the sequence must match the model's expected symbols.

In [15]:
from protein import Protein
from interaction import HPInteraction, MJInteraction
from constants import EMPTY_SIDECHAIN_PLACEHOLDER

# --- OPTION A: MJ Model (Miyazawa-Jernigan) ---
manual_interaction = MJInteraction()
seq = "APRLRFY"

# --- OPTION B: HP Model (Hydrophobic-Polar) ---
# manual_interaction = HPInteraction()

print(f"Using Model: {manual_interaction.__class__.__name__}")
print(f"Valid Symbols: {manual_interaction.valid_symbols}")

# Initialize Protein
manual_protein = Protein(
    main_protein_sequence=seq,
    side_protein_sequence=EMPTY_SIDECHAIN_PLACEHOLDER * len(seq),
    valid_symbols=manual_interaction.valid_symbols
)

Using Model: MJInteraction
Valid Symbols: {'D', 'S', 'H', 'I', 'A', 'M', 'F', 'P', 'T', 'W', 'N', 'C', 'G', 'K', 'V', 'Y', 'E', 'R', 'Q', 'L'}


### Step 2: Build Hamiltonian (Manually)

Here we manually instantiate the `ContactMap` and `DistanceMap`. These are computationally expensive pre-calculations that determine valid folding paths.

In [16]:
from contact import ContactMap
from distance import DistanceMap
from builder import HamiltonianBuilder
from utils.qubit_utils import remove_unused_qubits

manual_contact_map = ContactMap(manual_protein)
manual_distance_map = DistanceMap(manual_protein)

h_builder = HamiltonianBuilder(
    protein=manual_protein,
    interaction=manual_interaction,
    distance_map=manual_distance_map,
    contact_map=manual_contact_map,
)

full_h = h_builder.sum_hamiltonians()
compressed_h = remove_unused_qubits(full_h)

print(f"Manual Build Complete. Hamiltonian Size: {compressed_h.num_qubits} qubits")

Manual Build Complete. Hamiltonian Size: 9 qubits


### Step 3: Custom Solve

You can now plug this Hamiltonian into any Qiskit solver (Sampler, Estimator, VQE, QAOA).

In [17]:
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import real_amplitudes
from backend import get_sampler

# Setup Ansatz
ansatz = real_amplitudes(num_qubits=compressed_h.num_qubits, reps=1)
# Setup Optimizer
optimizer = COBYLA(maxiter=50)

# Get Sampler (Backend)
sampler, _ = get_sampler()

# Initialize VQE
vqe = SamplingVQE(
    sampler=sampler,
    ansatz=ansatz,
    optimizer=optimizer,
    aggregation=0.1
)

# Run
result = vqe.compute_minimum_eigenvalue(compressed_h)
print(f"Manual VQE Result: {result.eigenvalue}")

Manual VQE Result: -1.4249999999997274
